In [1]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


branca-0.3.1         | 25 KB     | #################################

## Introduction

The amount of crime in India is rising everyday. Crimes are categorized into multiple different groups. Our goal in this project is to figure out which states in the country has the most number of a certain crime. We finally conclude with which state has the most overall crimes.

## Data

The dataset is take from Kaggle. It contains information about the trials of violent crimes by courts and will eventually help determine the kind of results we watn. Foursquare data is leveraged in order to be able to plot our final results on a map.

In [61]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

body = client_55b2bb94579d4d9ba04aaf80a8aad6ca.get_object(Bucket='couseracapstone-donotdelete-pr-3vhu2vlrhbzpzg',Key='28_Trial_of_violent_crimes_by_courts.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

crimedf = pd.read_csv(body)
crimedf.head()

,Area_Name,Year,Group_Name,Sub_Group_Name,Trial_of_Violent_Crimes_by_Courts_By_Confession,Trial_of_Violent_Crimes_by_Courts_By_trial,Trial_of_Violent_Crimes_by_Courts_Total
0,Andhra Pradesh,2001,TVC- Arson,10. Arson,20.0,517.0,537.0
1,Arunachal Pradesh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0
2,Assam,2001,TVC- Arson,10. Arson,5.0,142.0,147.0
3,Bihar,2001,TVC- Arson,10. Arson,0.0,208.0,208.0
4,Chandigarh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0


In [62]:
crimedf.shape

(4473, 7)

In [63]:
crimedf.columns=['State', 'Year', 'Group', 'Sub_Group', 'Confession', 'Trial', 'Total']

In [64]:
crimedf.head(10)

,State,Year,Group,Sub_Group,Confession,Trial,Total
0,Andhra Pradesh,2001,TVC- Arson,10. Arson,20.0,517.0,537.0
1,Arunachal Pradesh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0
2,Assam,2001,TVC- Arson,10. Arson,5.0,142.0,147.0
3,Bihar,2001,TVC- Arson,10. Arson,0.0,208.0,208.0
4,Chandigarh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0
5,Chhattisgarh,2001,TVC- Arson,10. Arson,22.0,41.0,63.0
6,Dadra & Nagar Haveli,2001,TVC- Arson,10. Arson,NaN,2.0,2.0
7,Delhi,2001,TVC- Arson,10. Arson,18.0,75.0,93.0
8,Goa,2001,TVC- Arson,10. Arson,NaN,41.0,41.0
9,Gujarat,2001,TVC- Arson,10. Arson,2.0,228.0,230.0


In [65]:
crimedf.dropna(axis=0, inplace=True)
crimedf.shape

(3138, 7)

In [66]:
crimedf.drop(labels=['Group'], axis=1, inplace=True)
crimedf.shape

(3138, 6)

In [67]:
clist=crimedf.values.tolist()

In [76]:
#remove very small number of crimes
for i in range(len(clist)):
    if clist[i][5]<10:
        del clist[i]

In [77]:
len(clist)

2436

In [96]:
for i in range(len(clist)):
    if clist[i][2]=='12. Total Trials (Sum of 1-11 Above)':
        del clist[i]

In [97]:
len(clist)

2185

In [100]:
crimedf=pd.DataFrame(clist)
crimedf.columns=['State', 'Year', 'group', 'Confession', 'Trial', 'Total']
crimedf.head()

,State,Year,group,Confession,Trial,Total
0,Andhra Pradesh,2001,10. Arson,20.0,517.0,537.0
1,Assam,2001,10. Arson,5.0,142.0,147.0
2,Bihar,2001,10. Arson,0.0,208.0,208.0
3,Chhattisgarh,2001,10. Arson,22.0,41.0,63.0
4,Delhi,2001,10. Arson,18.0,75.0,93.0


In [102]:
crimedf.groupby('group').sum()

,Year,Confession,Trial,Total
group,,,,
01. Murder,415117,5077.0,189050.0,194127.0
02. Attempt to Murder,407086,2810.0,170023.0,172833.0
03. C H Not Amounting to Murder,292798,234.0,19743.0,19977.0
04. Rape,385041,2935.0,88255.0,91190.0
05. Kidnapping & Abduction,379045,1707.0,89898.0,91605.0
05A. Kidnapping & Abduction of Women & Girls,348935,1242.0,54407.0,55649.0
05B. Kidnapping & Abduction of Others,300823,461.0,24614.0,25075.0
06. Dacoity,298782,404.0,31222.0,31626.0
07. Preparation & Assembly for Dacoity,204550,123.0,10212.0,10335.0
